In [ ]:
import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from config import *
from Data.raw_data_loader import load_raw_crypto_csv
from Models.ou import build_spread, ou_signal_on_spread
from Backtest.metrics import build_summary_table, rolling_sharpe, sharpe_ratio
from Validation.monte_carlo import (
    iid_bootstrap_returns,
    block_bootstrap_returns,
    shuffled_sharpe_distribution,
    noise_injection_test,
    monte_carlo_report,
)

plt.style.use("default")

In [ ]:
df = load_raw_crypto_csv(DATA_PATH)
df = df.loc["2017-11-12":].copy()

btc = df["BTC-USD_close"].astype(float)
eth = df["ETH-USD_close"].astype(float)

df = df[(btc > 0) & (eth > 0)].copy()
print("Rows:", len(df), df.index.min(), df.index.max())

In [ ]:
def pair_portfolio_returns(df: pd.DataFrame, beta_window: int, position: pd.Series) -> pd.Series:
    log_b = np.log(df["BTC-USD_close"].astype(float))
    log_e = np.log(df["ETH-USD_close"].astype(float))

    minp = min(beta_window, max(30, beta_window // 3))
    cov = log_b.rolling(beta_window, min_periods=minp).cov(log_e)
    var = log_b.rolling(beta_window, min_periods=minp).var()
    beta = (cov / var).replace([np.inf, -np.inf], np.nan).fillna(method="ffill").fillna(0.0)

    r_btc = df["BTC-USD_close"].astype(float).pct_change().fillna(0.0)
    r_eth = df["ETH-USD_close"].astype(float).pct_change().fillna(0.0)

    position = position.reindex(df.index).fillna(0.0)
    pnl = position.shift(1).fillna(0.0) * (r_eth - beta * r_btc)
    return pnl


def apply_turnover_cost(returns: pd.Series, position: pd.Series, fee_bps: float, slippage_bps: float) -> pd.Series:
    pos = position.reindex(returns.index).fillna(0.0)
    turnover = pos.diff().abs().fillna(0.0)
    cost_rate = (fee_bps + slippage_bps) / 1e4
    return returns - turnover * cost_rate

In [ ]:
def walk_forward_splits(index: pd.DatetimeIndex, train_days: int, test_days: int, step_days: int):
    start = index.min()
    end = index.max()

    train_start = start
    train_end = start + pd.Timedelta(days=train_days)

    while True:
        test_start = train_end + pd.Timedelta(days=1)
        test_end = test_start + pd.Timedelta(days=test_days)
        if test_end > end:
            break

        yield (train_start, train_end, test_start, test_end)
        train_end = train_end + pd.Timedelta(days=step_days)


def simple_sharpe(r: pd.Series) -> float:
    r = r.dropna()
    if len(r) < 100:
        return np.nan
    vol = r.std()
    if vol == 0 or np.isnan(vol):
        return np.nan
    return float(np.sqrt(365) * r.mean() / vol)


param_grid = {
    "beta_window": [90, 180, 365],
    "ou_window": [60, 90, 180],
    "entry_z": [1.5, 2.0],
    "exit_z": [0.2, 0.3, 0.5],
}

train_days = 365 * 2
test_days = 180
step_days = 180

splits = list(walk_forward_splits(df.index, train_days, test_days, step_days))
print("Num folds:", len(splits))

fold_rows = []
all_test_gross = pd.Series(0.0, index=df.index)
all_test_net = pd.Series(0.0, index=df.index)
all_test_pos = pd.Series(0.0, index=df.index)

for k, (tr0, tr1, te0, te1) in enumerate(splits, 1):
    train_df = df.loc[tr0:tr1].copy()
    test_df = df.loc[te0:te1].copy()

    best = {"sharpe": -np.inf}

    for beta_w in param_grid["beta_window"]:
        spread_tr = build_spread(train_df, beta_window=beta_w)

        for ou_w in param_grid["ou_window"]:
            for entry in param_grid["entry_z"]:
                for exit_ in param_grid["exit_z"]:
                    pos_tr = ou_signal_on_spread(spread_tr, window=ou_w, entry_z=entry, exit_z=exit_)
                    pos_tr = pos_tr.reindex(train_df.index).fillna(0.0)

                    gross_tr = pair_portfolio_returns(train_df, beta_window=beta_w, position=pos_tr)
                    net_tr = apply_turnover_cost(gross_tr, pos_tr, fee_bps=FEE_BPS, slippage_bps=SLIPPAGE_BPS)

                    s = simple_sharpe(net_tr)
                    if np.isnan(s):
                        continue
                    if s > best["sharpe"]:
                        best = {
                            "fold": k,
                            "train_start": tr0,
                            "train_end": tr1,
                            "test_start": te0,
                            "test_end": te1,
                            "beta_window": beta_w,
                            "ou_window": ou_w,
                            "entry_z": entry,
                            "exit_z": exit_,
                            "train_sharpe": s,
                            "sharpe": s,
                        }

    beta_w = best["beta_window"]
    spread_te = build_spread(test_df, beta_window=beta_w)
    pos_te = ou_signal_on_spread(spread_te, window=best["ou_window"], entry_z=best["entry_z"], exit_z=best["exit_z"])
    pos_te = pos_te.reindex(test_df.index).fillna(0.0)

    gross_te = pair_portfolio_returns(test_df, beta_window=beta_w, position=pos_te)
    net_te = apply_turnover_cost(gross_te, pos_te, fee_bps=FEE_BPS, slippage_bps=SLIPPAGE_BPS)

    best["test_sharpe"] = simple_sharpe(net_te)

    fold_rows.append(best)
    all_test_gross.loc[test_df.index] = gross_te
    all_test_net.loc[test_df.index] = net_te
    all_test_pos.loc[test_df.index] = pos_te

wf_table = pd.DataFrame(fold_rows)
wf_table

In [ ]:
summary = build_summary_table({
    "WF Pair OU (Net)": {"returns": all_test_net.fillna(0.0), "position": all_test_pos.fillna(0.0)},
})

summary

In [ ]:
equity = (1.0 + all_test_net.fillna(0.0)).cumprod()

plt.figure(figsize=(12,5))
plt.plot(equity.index, equity.values)
plt.title("Walk-Forward Pair OU — Net Equity")
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(rolling_sharpe(all_test_net.fillna(0.0), window=365))
plt.title("Walk-Forward Pair OU — Rolling 365d Sharpe")
plt.show()

In [ ]:
r = all_test_net.dropna()
actual = sharpe_ratio(r)

iid_dist = iid_bootstrap_returns(r, n_samples=1000)
block_dist = block_bootstrap_returns(r, block_size=20, n_samples=1000)
shuffle_dist = shuffled_sharpe_distribution(r, n_samples=1000)

print("Actual Sharpe:", actual)
print("IID mean Sharpe:", float(np.nanmean(iid_dist)))
print("Block mean Sharpe:", float(np.nanmean(block_dist)))
print("Shuffle mean Sharpe:", float(np.nanmean(shuffle_dist)))

In [ ]:
plt.figure(figsize=(12,4))
plt.hist(iid_dist, bins=40)
plt.axvline(actual, linestyle="--")
plt.title("IID Bootstrap Sharpe Distribution")
plt.show()

plt.figure(figsize=(12,4))
plt.hist(block_dist, bins=40)
plt.axvline(actual, linestyle="--")
plt.title("Block Bootstrap Sharpe Distribution (block=20)")
plt.show()

plt.figure(figsize=(12,4))
plt.hist(shuffle_dist, bins=40)
plt.axvline(actual, linestyle="--")
plt.title("Shuffled Returns Sharpe Distribution")
plt.show()

In [ ]:
def prob_overfit(actual_sharpe, dist):
    return float(np.mean(dist >= actual_sharpe))

print("P(overfit) IID:", prob_overfit(actual, iid_dist))
print("P(overfit) Block:", prob_overfit(actual, block_dist))
print("P(overfit) Shuffle:", prob_overfit(actual, shuffle_dist))

In [ ]:
noise_dist = noise_injection_test(r, noise_std=0.001, n_samples=500)

plt.figure(figsize=(12,4))
plt.hist(noise_dist, bins=40)
plt.axvline(actual, linestyle="--")
plt.title("Noise Injection Sharpe Distribution (std=0.001)")
plt.show()

print("Noise mean Sharpe:", float(np.nanmean(noise_dist)))
print("P(noise >= actual):", float(np.mean(noise_dist >= actual)))

In [ ]:
def recompute_net_with_costs(fee_bps, slip_bps):
    net = apply_turnover_cost(all_test_gross, all_test_pos, fee_bps=fee_bps, slippage_bps=slip_bps)
    return net

cost_cases = {
    "0 bps": (0.0, 0.0),
    "10 bps": (10.0, 0.0),
    "20 bps": (10.0, 10.0),
}

rows = []
for name, (fee, slip) in cost_cases.items():
    net = recompute_net_with_costs(fee, slip)
    s = build_summary_table({name: {"returns": net.fillna(0.0), "position": all_test_pos}}).iloc[0].to_dict()
    rows.append({"CostCase": name, **s})

pd.DataFrame(rows).set_index("CostCase")

In [ ]:
wf_table[["fold","beta_window","ou_window","entry_z","exit_z","train_sharpe","test_sharpe"]]

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(wf_table["fold"], wf_table["test_sharpe"], marker="o")
plt.title("Test Sharpe by Fold")
plt.show()

plt.figure(figsize=(12,4))
plt.plot(wf_table["fold"], wf_table["train_sharpe"], marker="o")
plt.title("Train Sharpe by Fold")
plt.show()

In [ ]:
turnover = all_test_pos.diff().abs().fillna(0.0)
annual_turnover = float(turnover.sum() / len(turnover) * 365)

print("Annual turnover:", annual_turnover)

In [ ]:
report = monte_carlo_report(r, n_samples=1000, block_size=20)

robust_summary = {
    "ActualSharpe": report["ActualSharpe"],
    "ProbOverfit_Block": report["ProbOverfit_Block"],
    "ProbOverfit_Shuffle": report["ProbOverfit_Shuffle"],
    "AnnualTurnover": annual_turnover,
}

robust_summary